# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from sampler_utils import retrieve_autoencoder_components_state_dicts, latent_ids_to_onehot3d, get_latent_loaders
from models3d import VQAutoEncoder, Generator
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import torch.distributions as dists
from tqdm import tqdm
import gc
from models3d import BiomeClassifier



# FQVAE helper functions

In [4]:
import os
from log_utils import log, load_stats, load_model
import copy
from fq_models import FQModel, HparamsFQGAN


# Loads hparams from hparams.json file in saved model directory
def load_hparams_from_json(log_dir):
    import json
    import os
    json_path = os.path.join(log_dir, 'hparams.json')
    
    if not os.path.exists(json_path):
        raise FileNotFoundError(f"No hparams.json file found in {log_dir}")
    
    with open(json_path, 'r') as f:
        hparams = json.load(f)

    return hparams

# turns loaded hparams json into propery hyperparams object
def dict_to_vcqgan_hparams(hparams_dict, dataset=None):
    # Determine which hyperparameter class to use based on the dataset
    if dataset == None:
        dataset = hparams_dict.get('dataset', 'MNIST')  # Default to MNIST if not specified
    
    vq_hyper = HparamsFQGAN(dataset)
    # Set attributes from the dictionary
    for key, value in hparams_dict.items():
        setattr(vq_hyper, key, value)
    
    return vq_hyper

# Loads fqgan model weights from a given checkpoint file
def load_fqgan_from_checkpoint(H, fqgan):
    fqgan = load_model(fqgan, "fqgan", H.load_step, H.load_dir).cuda()
    fqgan.eval()
    return fqgan

# Takes a chunk or batch of chunks from the dataset, returns the encoded style and structure indices matrices
def encode_and_quantize(fqgan, terrain_chunks, device='cuda'):
    """Memory-efficient encoding function"""
    fqgan.eval()
    with torch.no_grad():
        # Move input to device
        terrain_chunks = terrain_chunks.to(device)
        
        # Get encodings
        h_style, h_struct = fqgan.encoder(terrain_chunks)
        
        # Process style path
        h_style = fqgan.quant_conv_style(h_style)
        quant_style, _, style_stats = fqgan.quantize_style(h_style)
        style_indices = style_stats[2]  # Get indices from tuple
        style_indices = style_indices.view(
            (h_style.size()[0], h_style.size()[2], h_style.size()[3], h_style.size()[4])
        )
        
        # Clear intermediate tensors
        del h_style, quant_style, style_stats
        
        # Process structure path
        h_struct = fqgan.quant_conv_struct(h_struct)
        quant_struct, _, struct_stats = fqgan.quantize_struct(h_struct)
        struct_indices = struct_stats[2]  # Get indices from tuple
        struct_indices = struct_indices.view(
            (h_struct.size()[0], h_struct.size()[2], h_struct.size()[3], h_struct.size()[4])
        )
        
        # Clear intermediate tensors
        del h_struct, quant_struct, struct_stats
        
        # Move indices to CPU to save GPU memory
        style_indices = style_indices.cpu()
        struct_indices = struct_indices.cpu()
        
        torch.cuda.empty_cache()
        
        return style_indices, struct_indices

# Takes style and structure indices, returns the reconstructed map
def decode_from_indices(style_indices, struct_indices, fqgan, device='cuda', two_stage=False, return_raw=False):
    """Memory-efficient decoding function"""
    with torch.no_grad():
        # Move indices to device only when needed
        style_indices = style_indices.to(device)
        struct_indices = struct_indices.to(device)
        
        # Get quantized vectors
        quant_style = fqgan.quantize_style.get_codebook_entry(
            style_indices.view(-1),
            shape=[1, fqgan.embed_dim, *style_indices.shape[1:]]
        )
        quant_struct = fqgan.quantize_struct.get_codebook_entry(
            struct_indices.view(-1),
            shape=[1, fqgan.embed_dim, *struct_indices.shape[1:]]
        )
        
        # Clear indices from GPU
        del style_indices, struct_indices
        
        # Combine and decode
        quant = torch.cat([quant_struct, quant_style], dim=1)
        # quant = quant_style + quant_struct
        del quant_style, quant_struct
        
        if two_stage:
            decoded, binary_decoded = fqgan.decoder(quant)
        else:
            decoded = fqgan.decoder(quant)
        
        del quant

        if return_raw:
            return decoded
        
        # Convert to block IDs if one-hot encoded
        if decoded.shape[1] > 1:
            decoded = torch.argmax(decoded, dim=1)
        
        # Move result to CPU and clear GPU memory
        result = decoded.squeeze(0).cpu()
        if two_stage:
            binary_result = binary_decoded.squeeze(0).cpu()
            del decoded
            del binary_decoded
            torch.cuda.empty_cache()
            return result, binary_result
        
        del decoded
        torch.cuda.empty_cache()
        
        return result

# Load FQ model

In [6]:
# replace with directory for whatever model you want
model_name = 'model_name'
model_path = f'model_dir/{model_name}'
model_checkpoint_step = 12000
fqgan_hparams =  dict_to_vcqgan_hparams(load_hparams_from_json(f"{model_path}"), 'minecraft')
fqgan_hparams.load_step = model_checkpoint_step
fqgan = FQModel(fqgan_hparams)
fqgan = load_fqgan_from_checkpoint(fqgan_hparams, fqgan)
print(f'loaded from: {fqgan_hparams.log_dir}')

using padding mode: reflect
With cycle consistency: True type: post_quant_conv, using gumbel: True
Using EMA quantizer
Using  TwoStageGenerator
Detaching binary reconstruction from comp graph for final loss
NO biome supervision
Disentangle Ratio:  0.5
Loading fqgan_12000.th
loaded from: FQGAN_2stagedecoder_logweighted3_1632codes_notdumb_0wd_nocc_withwater_vertflips


In [8]:
from data_utils import BlockBiomeConverter
mappings_path = '24_newdataset_mappings3.pt'
block_converter = BlockBiomeConverter.load_mappings(mappings_path)

In [9]:
# Create an instnance of our visualizer
from visualization_utils import MinecraftVisualizerPyVista
visualizer = MinecraftVisualizerPyVista()


# Four Corners Map

In [16]:
sample_name = 'flatgrass_low'
sample = torch.load(f'{sample_name}.pt')
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_0.png')
plotter.show()

y_min = 0
y_height = 3

close_corner = [(x, y, z) for x in [0, 1, 2] for y in  range(y_min, y_height) for z in  [3, 4, 5]]
far_corner = [(x, y, z) for x in [3, 4, 5] for y in  range(y_min, y_height) for z in  [0, 1, 2]]
left_corner = [(x, y, z) for x in [0, 1, 2] for y in  range(y_min, y_height) for z in  [0, 1, 2]]
# right_corner = [(x, y, z) for x in [3, 4, 5] for y in  range(y_min, y_height) for z in  [3, 4, 5]]
# right_corner = [(x, y, z) for x in [4] for y in  range(y_height, y_height + 2) for z in  [4, 5]]
right_corner_mountain = [(x, y, z) for x in [5] for y in  range(y_height, y_height + 2) for z in  [4, 5]]
right_corner_mountain += [(x, y, z) for x in [4] for y in  range(y_height, y_height + 2) for z in  [5]]
right_corner_slope = [(x, y, z) for x in [5] for y in  range(y_height, y_height + 2) for z in  [3]]
right_corner_slope += [(x, y, z) for x in [4] for y in  range(y_height, y_height + 2) for z in  [4]]
right_corner_slope += [(x, y, z) for x in [3] for y in  range(y_height, y_height + 2) for z in  [5]]
right_corner_slope += [(x, y, z) for x in [5] for y in  [y_height + 2] for z in  [5]]
right_corner = [(x, y, z) for x in [3] for y in  range(y_height, y_height + 3) for z in  [3, 4]]



close_corner_style_val = 4     # sand
far_corner_style_val = 3        # stone
left_corner_style_val = 2       # water
right_corner_style_val = 7     # dirt + grass

right_mountain_struct_val = 12
right_mountain_style_val = 3
right_slope_struct_val = 31
right_slope_style_val = 7
# swap close corner
for coord in close_corner:
    style_indices[0, coord[0], coord[1], coord[2]] = close_corner_style_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=close_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_1.png')
plotter.show()

# swap far corner
for coord in far_corner:
    style_indices[0, coord[0], coord[1], coord[2]] = far_corner_style_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=far_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_2.png')
plotter.show()

# swap left  corner
for coord in left_corner:
    style_indices[0, coord[0], coord[1], coord[2]] = left_corner_style_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=left_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_3.png')
plotter.show()


# swap right  corner
# build core out mountain out of stone
for coord in right_corner_mountain:
    style_indices[0, coord[0], coord[1], coord[2]] = right_mountain_style_val

for coord in right_corner_mountain:
    struct_indices[0, coord[0], coord[1], coord[2]] = right_mountain_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=left_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_mountaincore.png')
plotter.show()
# swap right  corner
# build slope of mountain out of grass
for coord in right_corner_slope:
    style_indices[0, coord[0], coord[1], coord[2]] = right_slope_style_val

for coord in right_corner_slope:
    struct_indices[0, coord[0], coord[1], coord[2]] = right_slope_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=left_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_mountainslope.png')
plotter.show()

# swap right  corner
for coord in right_corner:
    style_indices[0, coord[0], coord[1], coord[2]] = right_corner_style_val

for coord in right_corner:
    struct_indices[0, coord[0], coord[1], coord[2]] = 19

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=right_corner, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('fourcorner_4.png')
plotter.show()

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e75192ca60_35&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e818eba790_36&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e818eba850_37&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e81c91eb20_38&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e7e4b28cd0_39&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e7e4b72ca0_40&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:53056/index.html?ui=P_0x1e83a481e50_41&reconnect=auto" class="pyvi…

# Tunneling Experiments

## tunnel 1

In [110]:
sample_name = 'tunnelablemountain2'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=1, dims=(2, 4))
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('tunnel_exp_1_orig.png')
plotter.show()


tunnel_coords = [(x, y, z) for x in [4] for y in  [3] for z in  [1, 2, 3, 4]]
tunnel_coords_2 = [(x, y, z) for x in [2, 3, 4] for y in  [3] for z in  [1]]

tunnel_style_val = 1        # 1, 6, 7, 8 all seem to correspond to empty areas, start by leaving as is.
tunnel_struct_val = 9     # empty, 95.5% of the time

# swap close corner
for coord in tunnel_coords:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_struct_val



reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('tunnel1_firstleg.png')
plotter.show()


# swap close corner
for coord in tunnel_coords_2:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_struct_val



reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('tunnel1_secondtleg.png')
plotter.show()


second_tunnel_coords = [(x, y, z) for x in [0, 1, 2] for y in  [0] for z in  [3]]
second_tunnel_coords += [(x, y, z) for x in [2] for y in  [0] for z in  [3, 4, 5]]
second_tunnel_coords += [(3, 0, 2), (4, 0, 1), (5, 0, 0)]

# swap close corner
for coord in second_tunnel_coords:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_struct_val



reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)

reconstrcted_xray = np.copy(reconstruct_convert)
reconstrcted_xray[:, 4:, :] = 5
plotter = visualizer.visualize_chunk(reconstrcted_xray, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel1_secondtunnel_xray.png')
plotter.show()


Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x155f462d490_262&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1562bdeb760_263&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1564868e100_264&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1564fb77670_266&reconnect=auto" class="pyv…

In [38]:
sample_name = 'tunnelablemountain2'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=1, dims=(2, 4))
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel_exp_1_orig.png')
plotter.show()


tunnel_coords = [(x, y, z) for x in [4, 5] for y in  [2] for z in  [0, 1]]
tunnel_coords += [(3, 3, 3), (3, 3, 2), (3, 3, 1), (4, 3, 2), (2, 3, 2)]

tunnel_style_val = 1        # 1, 6, 7, 8 all seem to correspond to empty areas, start by leaving as is.
tunnel_struct_val = 9     # empty, 95.5% of the time

# swap close corner
for coord in tunnel_coords:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_struct_val



reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=tunnel_coords, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel1_alt.png')
plotter.show()


Widget(value='<iframe src="http://localhost:64481/index.html?ui=P_0x1ee926305b0_25&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:64481/index.html?ui=P_0x1ee97b757c0_26&reconnect=auto" class="pyvi…

## tunnel 2

In [55]:
import pyvista as pv


In [62]:
sample_name = 'snowstonemountain'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=2, dims=(2, 4))
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)

camera_elev_offset = -20

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel_exp_2_orig.png')
plotter.show()


tunnel_coords = [(x, y, z) for x in [3, 4] for y in  [0, 1] for z in  [1, 2, 3]]
tunnel_coords += [(x, y, z) for x in [4] for y in  [0, 1] for z in  [4]]
tunnel_coords += [(x, y, z) for x in [0, 1, 2] for y in  [0, 1] for z in  [3]]
# tunnel_coords += [(0, 1, 5), (1, 1, 4), (2, 1, 3)]
# tunnel_coords += [(0, 0, 5), (1, 0, 4), (2, 0, 3)]

tunnel_floor = [(x, y, z) for x in [3, 4] for y in  [0] for z in  [1, 2, 3, 4]]
tunnel_floor += [(x, y, z) for x in [0, 1, 2] for y in  [0] for z in  [3]]
# tunnel_coords_2 = [(x, y, z) for x in [2, 3, 4] for y in  [3] for z in  [1]]

tunnel_style_val = 1        # 1, 6, 7, 8 all seem to correspond to empty areas, start by leaving as is.
tunnel_struct_val = 9     # empty, 95.5% of the time
tunnel_floor_struct_val = 5 # floor
tunnel_floor_style_val = 6 # grass


# swap close corner
for coord in tunnel_coords:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_struct_val



reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel2_cave.png')
plotter.show()

reconstrcted_xray = np.copy(reconstruct_convert)
reconstrcted_xray[:, :, 16:] = 5
plotter = visualizer.visualize_chunk(reconstrcted_xray, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel2_cave_xray.png')
plotter.show()
# swap close corner
for coord in tunnel_floor:
    struct_indices[0, coord[0], coord[1], coord[2]] = tunnel_floor_struct_val
    style_indices[0, coord[0], coord[1], coord[2]] = tunnel_floor_style_val





reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel2_floor.png')
plotter.show()

reconstrcted_xray = np.copy(reconstruct_convert)
reconstrcted_xray[:, :, 16:] = 5
plotter = visualizer.visualize_chunk(reconstrcted_xray, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('tunnel2_floor_xray.png')
plotter.show()

reconstrcted_xray = np.copy(reconstruct_convert)
reconstrcted_xray[:, 8:, :] = 5
plotter = visualizer.visualize_chunk(reconstrcted_xray, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('tunnel2_firstleg.png')
plotter.show()

reconstrcted_xray = np.copy(reconstruct_convert)
reconstrcted_xray[:, :, 16:] = 5
reconstrcted_xray[:, 8:, :] = 5
plotter = visualizer.visualize_chunk(reconstrcted_xray, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('tunnel2_firstleg.png')
plotter.show()




Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x1325a2c3ac0_197&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x13388c319d0_198&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x1332a84fb20_199&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x13145a8a9d0_200&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x1317c0c4910_201&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x13197acb8e0_202&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:51129/index.html?ui=P_0x1319774c550_203&reconnect=auto" class="pyv…

# River Experiments

## river 1

In [11]:
sample_name = 'deserthill2'
sample = torch.load(f'{sample_name}.pt')
# sample = torch.rot90(sample, k=1, dims=(2, 4))
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_1_orig.png')
plotter.show()


river_coords = [(x, y, z) for x in [0, 1, 2, 3] for y in  [2, 3] for z in  [5]]
river_coords += [(x, y, z) for x in [3] for y in  [2, 3] for z in  [2, 3, 4, 5]]
river_coords += [(x, y, z) for x in [4, 5] for y in  [2, 3] for z in  [1]]

wide_river_coords = [(x, y, z) for x in [1, 2, 3] for y in  [2, 3] for z in  range(6)]
wide_river_coords += [(x, y, z) for x in [0] for y in  [2, 3] for z in  [3, 4, 5]]
wide_river_coords += [(x, y, z) for x in [4, 5] for y in  [2, 3] for z in  [0, 1]]

river_style_val = 2        # water style code
river_struct_val = 0     # empty, 95.5% of the time

riverbed_style_val = 15 

# swap close corner
for coord in river_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = river_style_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_1_river.png')
plotter.show()

# swap close corner
for coord in wide_river_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = river_style_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_1_wideriver.png')
plotter.show()

c:\Users\TimBits\miniconda3\envs\py39\lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x15297b87be0_0&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1530421cd60_1&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x152fc91e5e0_2&reconnect=auto" class="pyvis…

## River 2

In [60]:
sample_name = 'crazytreebowl'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=3, dims=(2, 4))
with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_2_orig.png')
plotter.show()


waterfall_coords = [(x, y, z) for x in [4] for y in  [2, 3, 4, 5] for z in  [3]]
waterfall_coords += [(x, y, z) for x in [4, 5] for y in  [5] for z in  [3]]
# waterfall += [(x, y, z) for x in [4, 5] for y in  [2, 3] for z in  [1]]

waterfall_stream_coords = [(x, y, z) for x in [3] for y in  [2] for z in  [3, 4]]
waterfall_stream_coords += [(2, 2, 4)]
# wide_river_coords += [(x, y, z) for x in [4, 5] for y in  [2, 3] for z in  [0, 1]]

# waterfall_ravine_coords = [(x, y, z) for x in [0, 1] for y in  [1, 2] for z in  [3]]
# waterfall_ravine_coords += [(x, y, z) for x in [1, 2] for y in  [1, 2] for z in  [4]]
# diagonal ravine
waterfall_ravine_coords = [(2, 1, 5), (2, 2, 5)]
waterfall_ravine_coords += [(1, 1, 4), (1, 2, 4)]
waterfall_ravine_coords += [(0, 1, 3), (0, 2, 3)]

# fill in cave
waterfall_ravine_coords += [(2, 0, 5), (1, 0, 5)]
waterfall_ravine_coords += [(3, 1, 5)]
waterfall_ravine_coords += [(4, 1, 5)]
waterfall_ravine_coords += [(4, 0, 5), (5, 0, 5)]



river_style_val = 2        # water style code
waterfall_struct_val = 4     # empty, 95.5% of the time

ravine_struct_val = 28 


# swap close corner
# for coord in waterfall_top_coords:
#     style_indices[0, coord[0], coord[1], coord[2]] = river_style_val
    # struct_indices[0, coord[0], coord[1], coord[2]] = waterfall_struct_val

for coord in waterfall_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = river_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = waterfall_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_2_waterfall.png')
plotter.show()


for coord in waterfall_stream_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = river_style_val
    # struct_indices[0, coord[0], coord[1], coord[2]] = waterfall_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_2_stream.png')
plotter.show()

for coord in waterfall_ravine_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = river_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = ravine_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_2_ravine.png')
plotter.show()


Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1535ecd8e20_127&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x15532e663a0_128&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x15393f86bb0_129&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x153826bc700_130&reconnect=auto" class="pyv…

In [10]:
def turn_latent_coords_into_val(array, latent_coords, val=999):
    for coord in latent_coords:
        x, y, z = coord
        x_min = x * 4
        x_max = x_min + 4
        y_min = y * 4
        y_max = y_min + 4
        z_min = z * 4
        z_max = z_min + 4
        array[x_min:x_max, y_min:y_max, z_min:z_max] = val

    return array
    

val = 166
area_viz = np.ones_like(reconstruct_convert)
area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=waterfall_coords, val=val)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.show()
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=waterfall_stream_coords, val=val)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.show()
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=waterfall_ravine_coords, val=val)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('river_exp_2_latentareas2.png')
plotter.show()


NameError: name 'reconstruct_convert' is not defined

# Floating Island Experiments

## floating island 1

In [ ]:
sample_name = 'tallmountain'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=2, dims=(2, 4))

camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island1_orig.png')
plotter.show()

# top layer

island_coords = [(1, 3, 1), (1, 4, 1)]
# island_coords = [(1, 2, 1), (1, 3, 1)]
# bottom layer
island_bottom = [(x, y, z) for x in [0, 1, 2] for y in  [4] for z in  [0, 1, 2]]
island_bottom.remove((1, 4, 1))
# island_bottom = [(x, y, z) for x in [0, 1, 2] for y in  [3] for z in  [0, 1, 2]]
# island_bottom.remove((1, 3, 1))

island_tree = [(x, y, z) for x in [0] for y in  [4, 5] for z in  [0]]
# island_tree.remove((1, 4, 1))
# waterfall_coords += [(x, y, z) for x in [4, 5] for y in  [5] for z in  [3]]

# waterfall_stream_coords = [(x, y, z) for x in [3] for y in  [2] for z in  [3, 4]]
# waterfall_stream_coords += [(2, 2, 4)]


island_style_val = 14        # water style code
island_struct_val = 28     # empty, 95.5% of the time

island_bottom_struct_val = 14
island_bottom_style_val = 13


island_tree_struct_val = 19
island_tree_style_val = 7
# 19 decent but blocky
# ravine_struct_val =  




for coord in island_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = island_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island1_core.png')
plotter.show()

for coord in island_bottom:
    style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_bottom_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island1_bottom.png')
plotter.show()

for coord in island_tree:
    style_indices[0, coord[0], coord[1], coord[2]] = island_tree_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_tree_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island1_tree.png')
plotter.show()



Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x15579a69700_248&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x155d08835e0_249&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1560c4c1f40_250&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x156108a5b50_251&reconnect=auto" class="pyv…

## Floating island 2

In [106]:
sample_name = 'tallmountain'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=2, dims=(2, 4))

camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island2_orig.png')
plotter.show()

# top layer

island_coords = [(1, 3, 1), (1, 4, 1)]
# island_coords = [(1, 2, 1), (1, 3, 1)]
# bottom layer
island_bottom = [(x, y, z) for x in [0, 1, 2] for y in  [4] for z in  [0, 1, 2]]
island_bottom.remove((1, 4, 1))
# island_bottom = [(x, y, z) for x in [0, 1, 2] for y in  [3] for z in  [0, 1, 2]]
# island_bottom.remove((1, 3, 1))

island_tree = [(x, y, z) for x in [0] for y in  [4, 5] for z in  [0]]
# island_tree.remove((1, 4, 1))
# waterfall_coords += [(x, y, z) for x in [4, 5] for y in  [5] for z in  [3]]

# waterfall_stream_coords = [(x, y, z) for x in [3] for y in  [2] for z in  [3, 4]]
# waterfall_stream_coords += [(2, 2, 4)]


island_style_val = 4        # water style code
island_struct_val = 28     # empty, 95.5% of the time

island_bottom_struct_val = 14
island_bottom_style_val = 4


island_tree_struct_val = 19
island_tree_style_val = 7
# 19 decent but blocky
# ravine_struct_val =  




for coord in island_coords:
    style_indices[0, coord[0], coord[1], coord[2]] = island_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island2_core.png')
plotter.show()

for coord in island_bottom:
    style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_bottom_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island2_bottom.png')
plotter.show()

for coord in island_tree:
    style_indices[0, coord[0], coord[1], coord[2]] = island_tree_style_val
    struct_indices[0, coord[0], coord[1], coord[2]] = island_tree_struct_val

reconstructed, _ = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('floating_island2_tree.png')
plotter.show()


Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x1559e493490_244&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x155d0883d00_245&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x155f46f7b80_246&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:54701/index.html?ui=P_0x15600a7c640_247&reconnect=auto" class="pyv…

In [26]:
sample_name = 'cutebeach2'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=2, dims=(2, 4))

# camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)


In [27]:
print(f'unique style codes: {torch.unique(style_indices, return_counts=True)}')
print(f'number of unique style codes: {len(torch.unique(style_indices))}')
print(f'unique struct codes: {torch.unique(struct_indices, return_counts=True)}')
print(f'number of unique style codes: {len(torch.unique(struct_indices))}')


unique style codes: (tensor([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15]), tensor([23, 32,  3, 11,  4, 62,  1, 16,  3,  6,  5,  5, 32, 13]))
number of unique style codes: 14
unique struct codes: (tensor([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 14, 15, 18, 22, 23, 24,
        25, 26, 27, 28, 29, 30]), tensor([ 3,  5, 10, 28,  1,  1, 11,  4,  7,  1, 15,  7,  1, 26,  7, 12,  5, 13,
         6,  3,  4, 12, 20, 14]))
number of unique style codes: 24


# Generative Experiment

## Generative Experiment 1

In [37]:
sample_name = 'cutebeach2'
sample = torch.load(f'{sample_name}.pt')
sample = torch.rot90(sample, k=2, dims=(2, 4))

# camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, binary_recon = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
binary_recon = (binary_recon>0.5).float()
binary_recon = binary_recon.squeeze()
binary_recon[binary_recon == 1] = 217
binary_recon[binary_recon == 0] = 5

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_1_orig.png')
plotter.show()




gen_struct_indices = torch.ones_like(struct_indices)
gen_style_indices = torch.ones_like(style_indices)

# starting val for style latents
all_style_val = 14
gen_style_indices = gen_style_indices * all_style_val

empty_style_val = 6



# structure changes
solid_coords = [(x, y, z) for x in range(6) for y in [0, 1, 2] for z in  range(6)]
slightly_elevated_coords =  [(x, y, z) for x in [2, 3] for y in  [3] for z in  [3, 4, 5]]
slightly_elevated_coords +=  [(x, y, z) for x in [4, 5] for y in  [3] for z in  [0, 1]]
slightly_elevated_coords +=  [(x, y, z) for x in [4, 5] for y in  [3] for z in  [5]]
hill_coords = [(x, y, z) for x in [4, 5] for y in  [3] for z in  [2, 3, 4]]

empty_air_coords = [(x, y, z) for x in range(6) for y in [3, 4, 5] for z in  range(6)]
# empty_air_coords += [(x, y, z) for x in [0, 1] for y in [3] for z in  range(6)]
# empty_air_coords += [(x, y, z) for x in [2, 3] for y in [3] for z in  [0, 1, 2]]

# bottom layer water
water_style_coords = [(x, y, z) for x in [0] for y in [0] for z in  [0, 1, 2]]
water_style_coords += [(1, 0, 0)]

# second layer water
water_style_coords += [(x, y, z) for x in [0] for y in [1] for z in  range(6)]
water_style_coords += [(x, y, z) for x in [1] for y in [1] for z in  [0, 1, 2]]
water_style_coords += [(2, 1, 0)]

# top layer water
water_style_coords += [(x, y, z) for x in [0, 1] for y in [2] for z in  range(6)]
water_style_coords += [(x, y, z) for x in [2] for y in [2] for z in  [0, 1, 2]]
water_style_coords += [(3, 2, 0)]


# first layer sand
sand_style_coords = [(x, y, z) for x in [1] for y in [0, 1] for z in [3, 4, 5]]
sand_style_coords += [(x, y, z) for x in [2] for y in [0, 1] for z in [1, 2, 3]]
sand_style_coords += [(x, y, z) for x in [3] for y in [0, 1] for z in [0, 1]]

# second layer sand
sand_style_coords += [(x, y, z) for x in [2] for y in [2] for z in [3, 4, 5]]
sand_style_coords += [(x, y, z) for x in [3] for y in [2] for z in [1, 2, 3, 4, 5]]
sand_style_coords += [(x, y, z) for x in [4, 5] for y in [2] for z in [0]]

# bottom layer sand
sand_style_coords += [(x, y, z) for x in [0] for y in [0] for z in [4, 5]]

# grass style coords
grass_style_coords = [(x, y, z) for x in [4, 5] for y in [2, 3] for z in [1, 2, 3, 4, 5]]
# grass_style_coords += [(x, y, z) for x in [3] for y in [2] for z in [1, 2, 3, 4, 5]]

#

solid_struct_val = 29   # solid chunk
slightly_elevated_struct_val = 0    # bottom layer filled
hill_struct_val = 30

water_style_val = 2
sand_style_val = 4
grass_style_val = 12

for coord in empty_air_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = empty_style_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
# pil_frame.save('generative_1_sand.png')
# plotter.show()

for coord in solid_coords:
    # gen_style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = solid_struct_val

for coord in slightly_elevated_coords:
    # gen_style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = slightly_elevated_struct_val

for coord in hill_coords:
    # gen_style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = hill_struct_val

for coord in water_style_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = water_style_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)
binary_recon = (binary_recon>0.5).float()
binary_recon = binary_recon.squeeze()
binary_recon[binary_recon == 1] = 217
binary_recon[binary_recon == 0] = 5

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_1_water.png')
# plotter.show()

plotter = visualizer.visualize_chunk(binary_recon, interactive=True, show_axis=False, highlight_latents=None, wireframe_highlight=True)
# plotter.camera.elevation += camera_elev_offset
# plotter.add_light(pv.Light(light_type='headlight', intensity=0.2))
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_1_struct_bin_recon.png')
# plotter.show()

for coord in sand_style_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = sand_style_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_1_sand.png')
# plotter.show()

for coord in grass_style_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = grass_style_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_1_grass.png')
plotter.show()



# build solid base
# gen_struct_indices[:, :3, :] = 29

# add small hill in grassy area


Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x1450e525580_153&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x14557ff1f70_158&reconnect=auto" class="pyv…

In [ ]:
solid_coords = [(x, y, z) for x in range(6) for y in [0, 1, 2] for z in  range(6)]

In [38]:
area_viz = np.ones_like(reconstruct_convert)
area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=solid_coords, val=217)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_solid_struct_coords.png')
plotter.show()

area_viz = turn_latent_coords_into_val(area_viz, latent_coords=slightly_elevated_coords, val=95)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_slightly_elevated_struct_coords.png')
plotter.show()


area_viz = turn_latent_coords_into_val(area_viz, latent_coords=hill_coords, val=250)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_hill_struct_coords.png')
plotter.show()

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x1454b4dff10_159&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x144f653efa0_160&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x1448ae88fa0_161&reconnect=auto" class="pyv…

In [39]:
val = 166
area_viz = np.ones_like(reconstruct_convert)
area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=solid_coords, val=217)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_solid_vs_empty_style_coords.png')
plotter.show()

# area_viz = np.ones_like(reconstruct_convert)
# area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=water_style_coords, val=240)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_water_style_coords.png')
plotter.show()


# area_viz = np.ones_like(reconstruct_convert)
# area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=sand_style_coords, val=194)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_sand_style_coords.png')
plotter.show()


area_viz = turn_latent_coords_into_val(area_viz, latent_coords=grass_style_coords, val=93)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_grass_style_coords.png')
plotter.show()

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x14324ad66a0_162&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x14463f42070_163&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x141c67809a0_164&reconnect=auto" class="pyv…

Widget(value='<iframe src="http://localhost:59745/index.html?ui=P_0x141e5ed1790_165&reconnect=auto" class="pyv…

## Generative Experiment 2

In [22]:
sample_name = 'grassbowl'
sample = torch.load(f'{sample_name}.pt')
# sample = torch.rot90(sample, k=2, dims=(2, 4))

# camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)

In [25]:
print(f'unique style codes: {torch.unique(style_indices, return_counts=True)}')
print(f'number of unique style codes: {len(torch.unique(style_indices))}')
print(f'unique struct codes: {torch.unique(struct_indices, return_counts=True)}')
print(f'number of unique style codes: {len(torch.unique(struct_indices))}')


unique style codes: (tensor([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), tensor([66,  2,  1, 20,  4, 49,  1, 14,  8,  4,  7,  9, 20, 10,  1]))
number of unique style codes: 15
unique struct codes: (tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18, 20,
        22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), tensor([12,  9, 17,  2,  1,  3,  3,  6,  4,  4,  8, 12,  2,  1, 32,  1, 32,  1,
         9,  6,  9,  4,  1,  2,  5, 12, 15,  3]))
number of unique style codes: 28


In [18]:
sample_name = 'grassbowl'
sample = torch.load(f'{sample_name}.pt')
# sample = torch.rot90(sample, k=2, dims=(2, 4))

# camera_elev_offset = -20

with torch.no_grad():
    style_indices, struct_indices = encode_and_quantize(fqgan, sample)
reconstructed, binary_recon = decode_from_indices(style_indices, struct_indices, fqgan, two_stage=True)
binary_recon = (binary_recon>0.5).float()
binary_recon = binary_recon.squeeze()
binary_recon[binary_recon == 1] = 217
binary_recon[binary_recon == 0] = 5

reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
# plotter.camera.elevation += camera_elev_offset
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_2_orig.png')
plotter.show()



gen_struct_indices = torch.ones_like(struct_indices)
gen_style_indices = torch.ones_like(style_indices)

# starting val for style latents
all_style_val = 3
gen_style_indices = gen_style_indices * all_style_val




# structure changes
solid_coords = [(x, y, z) for x in range(6) for y in [0, 1, 2] for z in  range(6)]

bowl_rim_coords = [(x, y, z) for x in range(6) for y in  [3, 4] for z in  [0]]
bowl_rim_coords += [(x, y, z) for x in [5] for y in  [3, 4] for z in  range(6)]

rim_slope_coords = [(x, y, z) for x in [0, 1, 2, 3, 4] for y in  [2, 3] for z in  [1]]
rim_slope_coords += [(x, y, z) for x in [4] for y in  [2, 3] for z in  [1, 2, 3, 4, 5]]
rim_slope_coords += [(x, y, z) for x in [4] for y in  [2, 3, 4] for z in  [1]]
rim_slope_coords += [(x, y, z) for x in [3] for y in  [2, 3] for z in  [2]]

rim_top_coords = [(4, 5, 0), (5, 5, 0), (5, 5, 1)]


empty_air_coords = [(x, y, z) for x in [0, 1, 2, 3] for y in [3, 4, 5] for z in [2, 3, 4, 5]]
empty_air_coords += [(x, y, z) for x in [0, 1, 2, 3] for y in [5] for z in [0]]
empty_air_coords += [(x, y, z) for x in [5] for y in [5] for z in [2, 3, 4, 5]]


# # grass style coords
grass_style_coords = [(x, y, z) for x in [0, 1, 2, 3] for y in [2, 3] for z in [2, 3, 4, 5]]
grass_style_coords += [(x, y, z) for x in [0, 1, 2, 3] for y in [4, 5] for z in [0]]
grass_style_coords += [(x, y, z) for x in [5] for y in [4, 5] for z in [2, 3, 4, 5]]
grass_style_coords += [(4, 5, 0), (5, 5, 0), (5, 5, 1)]
grass_style_coords +=  [(x, y, z) for x in [0, 1, 2, 3, 4] for y in  [3, 4] for z in  [1]]
grass_style_coords += [(x, y, z) for x in [4] for y in  [3, 4] for z in  [1, 2, 3, 4, 5]]
grass_style_coords += [(x, y, z) for x in [4] for y in  [2, 3, 4] for z in  [1]]
grass_style_coords += [(x, y, z) for x in [3] for y in  [2, 3] for z in  [2]]
#

solid_struct_val = 29   # solid chunk
rim_val = 29    # bottom layer filled
rim_top_val = 0
rim_slope_val = 30

rim_slope_style_val = 6
rim_top_style_val = 6
rim_style_val = 6

water_style_val = 2
sand_style_val = 4
grass_style_val = 6

empty_style_val = 6


for coord in empty_air_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = empty_style_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)



for coord in solid_coords:
    # gen_style_indices[0, coord[0], coord[1], coord[2]] = island_bottom_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = solid_struct_val

for coord in bowl_rim_coords:
    # gen_style_indices[0, coord[0], coord[1], coord[2]] = rim_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = rim_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)

print("solid + rim struct")
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_2_rim.png')
# plotter.show()

# 
for coord in rim_top_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = grass_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = rim_top_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)
print("rim top struct")
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_2_rim_top.png')
# plotter.show()



for coord in rim_slope_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = grass_style_val
    gen_struct_indices[0, coord[0], coord[1], coord[2]] = rim_slope_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)
print("rim slope struct")
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_2_rim_slope.png')
# plotter.show()

for coord in grass_style_coords:
    gen_style_indices[0, coord[0], coord[1], coord[2]] = grass_style_val
    # gen_struct_indices[0, coord[0], coord[1], coord[2]] = rim_top_val

reconstructed, binary_recon = decode_from_indices(gen_style_indices, gen_struct_indices, fqgan, two_stage=True)
print('grass style')
reconstruct_convert = block_converter.convert_to_original_blocks(reconstructed.squeeze())
plotter = visualizer.visualize_chunk(reconstruct_convert, interactive=True, show_axis=False)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative_2_grass.png')
plotter.show()


#


Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17d1f9971f0_24&reconnect=auto" class="pyvi…

solid + rim struct
rim top struct
rim slope struct
grass style


Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17d45510880_28&reconnect=auto" class="pyvi…

In [12]:
area_viz = np.ones_like(reconstruct_convert)
area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=solid_coords, val=217)
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=bowl_rim_coords, val=217)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative2_solid_struct_coords.png')
plotter.show()

area_viz = turn_latent_coords_into_val(area_viz, latent_coords=rim_top_coords, val=95)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative2_rimtop_struct_coords.png')
plotter.show()


area_viz = turn_latent_coords_into_val(area_viz, latent_coords=rim_slope_coords, val=250)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative2_slope_struct_coords.png')
plotter.show()

Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17cfd8fd7f0_5&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17cfd9737c0_6&reconnect=auto" class="pyvis…

Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17cfd8fc100_7&reconnect=auto" class="pyvis…

In [19]:
val = 166
area_viz = np.ones_like(reconstruct_convert)
area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=solid_coords, val=217)
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=bowl_rim_coords, val=217)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative2_solid_vs_empty_style_coords.png')
plotter.show()

# area_viz = np.ones_like(reconstruct_convert)
# area_viz = area_viz * 5
print(grass_style_coords)
val = 166
# area_viz = np.ones_like(reconstruct_convert)
# area_viz = area_viz * 5
area_viz = turn_latent_coords_into_val(area_viz, latent_coords=grass_style_coords, val=93)
plotter = visualizer.visualize_chunk(area_viz, interactive=True, show_axis=True, highlight_latents=None, wireframe_highlight=True)
frame = plotter.screenshot(transparent_background=True, return_img=True)
pil_frame = Image.fromarray(frame)
pil_frame.save('generative2_grass_style_coords.png')
plotter.show()


Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17d32c68b50_29&reconnect=auto" class="pyvi…

[(0, 2, 2), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 3, 2), (0, 3, 3), (0, 3, 4), (0, 3, 5), (1, 2, 2), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 2), (1, 3, 3), (1, 3, 4), (1, 3, 5), (2, 2, 2), (2, 2, 3), (2, 2, 4), (2, 2, 5), (2, 3, 2), (2, 3, 3), (2, 3, 4), (2, 3, 5), (3, 2, 2), (3, 2, 3), (3, 2, 4), (3, 2, 5), (3, 3, 2), (3, 3, 3), (3, 3, 4), (3, 3, 5), (0, 4, 0), (0, 5, 0), (1, 4, 0), (1, 5, 0), (2, 4, 0), (2, 5, 0), (3, 4, 0), (3, 5, 0), (5, 4, 2), (5, 4, 3), (5, 4, 4), (5, 4, 5), (5, 5, 2), (5, 5, 3), (5, 5, 4), (5, 5, 5), (4, 5, 0), (5, 5, 0), (5, 5, 1), (0, 3, 1), (0, 4, 1), (1, 3, 1), (1, 4, 1), (2, 3, 1), (2, 4, 1), (3, 3, 1), (3, 4, 1), (4, 3, 1), (4, 4, 1), (4, 3, 1), (4, 3, 2), (4, 3, 3), (4, 3, 4), (4, 3, 5), (4, 4, 1), (4, 4, 2), (4, 4, 3), (4, 4, 4), (4, 4, 5), (4, 2, 1), (4, 3, 1), (4, 4, 1), (3, 2, 2), (3, 3, 2)]


Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17d1f9be5b0_30&reconnect=auto" class="pyvi…

A view with name (P_0x17d1f9be5b0_30) is already registered
 => returning previous one


Widget(value='<iframe src="http://localhost:63689/index.html?ui=P_0x17d1f9be5b0_30&reconnect=auto" class="pyvi…